### Building a Simple LLM Application with LCEL (LangChain Expression Language)
##### This application will translate text from English to another language. This is just a single LLM call using some prompting.
##### This application gives overview of:
* Using language models 
* Using PromptTemplates and StrOutputParsers
* Using LangChain Expression Language (LCEL) to chain components together 
* Debugging and tracing application with LangSmith
* Deploying application with LangServe

### Why Groq?
* Groq is the AI infrastructure company that delivers fast AI inference
* The LPU (Large Processing Unit) Inference Engine by Groq is a hardware and software platform that delivers exceptional compute speed,
quality, and energy efficient
* An LPU is Groq’s custom chip designed specifically for AI inference, especially large language models (LLMs).
* Unlike GPUs (many cores doing many things), the LPU uses a deterministic, single-core–style architecture, so every operation happens in a predictable order.
* LPU is designed to overcome the two LLM bottlenecks: compute density and memory bandwidth. A LPU has greater compute capacity than a GPU & CPU in regards to LLMs. This reduces the amount of time per word calculated, allowing sequence of text to be generated must faster.Additionally, eliminating external memory bottlenecks enables the LPU Inference Engine to deliver orders of magnitude better performance on LLMs compared to GPUs

In [5]:
### Open AI API key and Open Source Models - Llama3,Gemma2, Mistral on platform Groq 

import os 
from dotenv import load_dotenv
load_dotenv()  # this will load all the keys from ".env" file

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

# for groq
groq_api_key=os.getenv("GROQ_API_KEY")

# installations 
# !pip install langchain_groq
# !pip install langchain_core


In [8]:
### import ChatGoq

from langchain_groq import ChatGroq
model=ChatGroq(model="groq/compound",groq_api_key=groq_api_key)
model

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x00000237C86AAF00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000237C8E41FD0>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain.messages import HumanMessage,SystemMessage
# systemmessage is how LLM model should be behave/instruct. In this case, it behaves like a translator
# HumanMessage send to LLM and get the response. In another way - what sentence I need to convert?
"""
messages = [
            SystemMessage(content="Translate the following from English to French"), 
            HumanMessage(content="Hello,What is time now?"),
        ]

result=model.invoke(messages)
result
"""

'\nmessages = [\n            SystemMessage(content="Translate the following from English to French"), \n            HumanMessage(content="Hello,What is time now?"),\n        ]\n\nresult=model.invoke(messages)\nresult\n'

In [29]:
# Testing from English to Spanish

messages = [
            SystemMessage(content="Translate the following from English to Spanish"), 
            HumanMessage(content="How are you?"),
        ]

result=model.invoke(messages)
result


AIMessage(content='**¿Cómo estás?**', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 259, 'total_tokens': 320, 'completion_time': 0.131116, 'completion_tokens_details': None, 'prompt_time': 0.010973, 'prompt_tokens_details': None, 'queue_time': 0.477865, 'total_time': 0.142088}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4178-e942-7933-90d7-12dba6721f77-0', usage_metadata={'input_tokens': 259, 'output_tokens': 61, 'total_tokens': 320})

In [ ]:
# StrOutputParser display custom or AI message and convert into String format (discard all other details)

from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'**¿Cómo estás?**'

In [ ]:
### using LCEL - we can chain (one after the other) the components
### chain=model|parser ==> when invoke() method executed first messages will pass to the model and get the result and that result
### pass to parser to convert StrOutputParser

chain=model|parser
chain.invoke(messages)  # messages = SystemMessage,HumanMessage

'¿Cómo estás?'

In [ ]:
### Prompt Templates - instead of passing many messages individually, create a prompt template to make it generic

from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)


In [35]:
result=prompt.invoke({"language":"Spanish","text":"Hello"})

In [ ]:
result.to_messages() 

[SystemMessage(content='Translate the following into Spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [ ]:
# Chaining togethter components with LCEL

chain=prompt|model|parser
chain.invoke({"language":"Telugu","text":"My name is Venkat"})

'నా పేరు వెంకట్.'

In [41]:
chain.invoke({"language":"French","text":"Hello"})

'Bonjour'